This file evaluates how well the BERT model trained with supervised learning for 3 classes does with manually labelled molecules from the wet lab team 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
bert_scores_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/bert_regen_score_dict_supervised_3_classes_no_molecules.csv")

In [ ]:
bert_scores_df

,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
0,iopn,0.046197,0.697341,0.256462,1
1,igf-i,0.003653,0.091289,0.905058,1
2,gfra,0.244903,0.748698,0.006400,4
3,adam,0.237668,0.185988,0.576344,5
4,mirna-21,0.999660,0.000159,0.000181,1
...,...,...,...,...,...
817,sema3a,0.846462,0.036098,0.117440,240
818,igf1,0.543830,0.240735,0.215435,11
819,nogo66,0.794675,0.089106,0.116219,10
820,shh,0.462577,0.200123,0.337300,79


In [ ]:
CK_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/CK_MoleculesToLabel_JJ.xlsx")
AH_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AH_MoleculesLabeled.xlsx")
AC_molecules = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/NLP - Lab/GraphSage/ManuallyLabeledMolecules/AC_MoleculesToLabel_JJ.xlsx")

In [ ]:
# Ensure all columns have same rows 
mask_CK = bert_scores_df.iloc[:,0] == CK_molecules.iloc[:,0]
mask_AC = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]
mask_AH = bert_scores_df.iloc[:,0] == AH_molecules.iloc[:,0]

print(list(mask_AH) == list(mask_CK))
print(list(mask_AC) == list(mask_CK))
print(list(mask_AH) == list(mask_AC))

True
True
True


In [ ]:
# These are the molecules that need to be relabeled since greek letters 
# didn't get converted properly 
molecules_to_relabel_BERT = list(bert_scores_df[mask_CK == False].iloc[:,0])
molecules_to_relabel_excel = list(CK_molecules[mask_CK == False].iloc[:,0])
bert_scores_df[mask_CK == False]

,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
10,ifnγ,0.875249,0.123310,0.001441,5
38,tgf-β,0.478886,0.175836,0.345278,15
52,tnf-α,0.314417,0.388922,0.296661,18
57,gsk3β,0.699657,0.194657,0.105686,89
75,gsk-3β,0.942369,0.056762,0.000868,13
122,gmf-,0.112389,0.456985,0.430626,4
133,gad- d45α,0.999442,0.000225,0.000333,1
215,tgf-(1,0.001200,0.024280,0.974519,1
311,pkcα,0.051267,0.946319,0.002414,2
317,gfap –/–,0.996635,0.003158,0.000208,1


In [ ]:
# Check if any duplicate labeled molecules across sheets 
CK_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AH_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index
AC_indeces = CK_molecules.loc[:,"Promoter/Inhibtior"][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False].index

# Only 1 and 4 are common among all sheets which is fine since that must have been pre-labeled 
print(set(CK_indeces) & set(AH_indeces))
print(set(CK_indeces) & set(AC_indeces))
print(set(AC_indeces) & set(AH_indeces))

{1, 4}
{1, 4}
{1, 4}


In [ ]:
CK_molecules.loc[:,"Abbreviated Name"]

0          iopn
1         igf-i
2          gfra
3          adam
4      mirna-21
         ...   
817      sema3a
818        igf1
819      nogo66
820         shh
821         atf
Name: Abbreviated Name, Length: 822, dtype: object

In [ ]:
# Concatenate the non-null values of all excel sheets 
non_null_CK = CK_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][CK_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AC = AC_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AC_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]
non_null_AH = AH_molecules.loc[:,["Abbreviated Name", "Promoter/Inhibtior"]][AH_molecules.loc[:,"Promoter/Inhibtior"].isnull() == False]

# Validate by ensuring lenghth adds up 
print(len(non_null_CK) + len(non_null_AC) + len(non_null_AH))

all_man_labeled = pd.concat([non_null_CK, non_null_AC, non_null_AH])
print(len(all_man_labeled))

141
141


,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
2,gfra,0.244903,0.748698,0.0064,4


In [ ]:
conversion_dict = {'i': 0, 'p': 1, 'n': 2}

mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == "gapdh"]
conversion_dict[mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()]

1

In [ ]:
all_man_mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == "gapdh"]
first_letter = all_man_mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()
conversion_dict[first_letter]

1

In [ ]:
bert_scores_df[bert_scores_df.iloc[:,0] == "fgf-2"]

,Unnamed: 0,% Inhibitor,% Promoter,% Neither,Total Sentences Found
703,fgf-2,0.130378,0.57921,0.290412,227


In [ ]:
conversion_dict = {'i': 0, 'p': 1, 'n': 2}
# Ensure labeling is standardized across sheets 
list(all_man_labeled.loc[:,"Promoter/Inhibtior"])

# Variation in labeling so just use first letter for Promoter/Inhibitor/Neither
# accepted_values = ["promoter", "inhibitor", "neither", "unknown", "error"]

# track breakdown of wrong results 
labeled_promoter_as_inhibitor = 0 
labeled_inhibitor_as_promoter = 0
labeled_promoter_as_neither = 0
labeled_inhibitor_as_neither = 0 
labeled_neither_as_promoter = 0 
labeled_neither_as_inhibitor = 0 

# track overall acurracy 
correct = 0
total = 0
for molecule in list(all_man_labeled.loc[:,"Abbreviated Name"]):
    # skip greek letter ones to be relabeled 
    if molecule in molecules_to_relabel_excel: 
      print("{} skipped".format(molecule))
      continue 
 
    BERT_mol_row = bert_scores_df[bert_scores_df.iloc[:,0] == molecule]
    BERT_mol_label = np.argmax(BERT_mol_row.values.flatten().tolist()[1:4]) # 0: Inhib, 1: Prom, 2: Neith  
    
    all_man_mol_row = all_man_labeled[all_man_labeled.iloc[:,0] == molecule]
    first_letter = all_man_mol_row.loc[:,"Promoter/Inhibtior"].values[0][0].lower()
    # Skip if not standardized response
    if first_letter not in conversion_dict:
      continue 

    all_man_mol_label = conversion_dict[first_letter]

    if BERT_mol_label == all_man_mol_label:
      correct += 1
    elif BERT_mol_label == 0 and all_man_mol_label == 1:
      labeled_promoter_as_inhibitor += 1
    elif BERT_mol_label == 1 and all_man_mol_label == 0:
      labeled_inhibitor_as_promoter += 1
    elif BERT_mol_label == 2 and all_man_mol_label == 1:
      labeled_promoter_as_neither += 1
    elif BERT_mol_label == 2 and all_man_mol_label == 0:
      labeled_inhibitor_as_neither += 1
    elif BERT_mol_label == 1 and all_man_mol_label == 2:
      labeled_neither_as_promoter += 1
    elif BERT_mol_label == 0 and all_man_mol_label == 2:
      labeled_neither_as_inhibitor += 1

    total += 1
    print("{}: MAN_LABELED: {}, BERT: {}".format(molecule, all_man_mol_label, BERT_mol_label))

In [ ]:
print("Correct/Total: {}/{} ({:.2f})".format(correct, total, correct/total)) 
incorrects = total - correct 

print("Labeled promoter as inhibitor: {:.2f}".format(labeled_promoter_as_inhibitor/incorrects)) 
print("Labeled inhibitor as promoter: {:.2f}".format(labeled_inhibitor_as_promoter/incorrects)) 
print("Labeled promoter as neither: {:.2f}".format(labeled_promoter_as_neither/incorrects)) 
print("Labeled inhibitor as neither: {:.2f}".format(labeled_inhibitor_as_neither/incorrects)) 
print("Labeled neither as promoter: {:.2f}".format(labeled_neither_as_promoter/incorrects)) 
print("Labeled neither as inhibitor: {:.2f}".format(labeled_neither_as_inhibitor/incorrects)) 

# Validation 
incorrects == sum([labeled_promoter_as_inhibitor, labeled_inhibitor_as_promoter, labeled_promoter_as_neither, labeled_inhibitor_as_neither, labeled_neither_as_promoter, labeled_neither_as_inhibitor])

Correct/Total: 45/131 (0.34)
Labeled promoter as inhibitor: 0.38
Labeled inhibitor as promoter: 0.12
Labeled promoter as neither: 0.22
Labeled inhibitor as neither: 0.07
Labeled neither as promoter: 0.07
Labeled neither as inhibitor: 0.14


True